In [17]:
import pandas as pd
import numpy as np
import chromadb
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.output_parsers import CommaSeparatedListOutputParser
from semantic_router.layer import RouteLayer
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

#BERTscore
import bert_score
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)
from bert_score import BERTScorer
from BRAD import brad
from BRAD import rag

## Single Document Calling

In [2]:
path = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/'
vectordb, embeddings_model = brad.load_literature_db(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/')

Wed Jun 19 09:34:53 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/machoi/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Wed Jun 19 09:35:07 2024 INFO Use pytorch device_name: cpu
Wed Jun 19 09:35:08 2024 INFO Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Wed Jun 19 09:35:09 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.


In [3]:
help(vectordb)

Help on Chroma in module langchain_chroma.vectorstores object:

class Chroma(langchain_core.vectorstores.VectorStore)
 |  Chroma(collection_name: 'str' = 'langchain', embedding_function: 'Optional[Embeddings]' = None, persist_directory: 'Optional[str]' = None, client_settings: 'Optional[chromadb.config.Settings]' = None, collection_metadata: 'Optional[Dict]' = None, client: 'Optional[chromadb.ClientAPI]' = None, relevance_score_fn: 'Optional[Callable[[float], float]]' = None, create_collection_if_not_exists: 'Optional[bool]' = True) -> 'None'
 |  
 |  `ChromaDB` vector store.
 |  
 |  To use, you should have the ``chromadb`` python package installed.
 |  
 |  Example:
 |      .. code-block:: python
 |  
 |              from langchain_chroma import Chroma
 |              from langchain_openai import OpenAIEmbeddings
 |  
 |              embeddings = OpenAIEmbeddings()
 |              vectorstore = Chroma("langchain_store", embeddings)
 |  
 |  Method resolution order:
 |      Chroma
 | 

In [4]:
test_list = list(vectordb.get()['ids'][:10])

In [5]:
source_title = []
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
print(path)
for metadata in vectordb.get()['metadatas']:
    source_title.append(metadata['source'].removeprefix(path).removesuffix(".pdf"))
no_repeat_source_title = set(source_title)
post_process_title = [x.lower() for x in no_repeat_source_title]
print(post_process_title)

/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/
['a transient disruption of fibroblastic', 'szklarczyk2023string', 'in vivo cellular reprogramming the next generation', 'direct cell reprogramming', 'ismara automated modeling of genomic signals as a democracy of regulatory motifs', 'similar active genes cluster in specialized transcription factories', 'cooperative transcription factor induction', 'updated perspectives on direct vascular cellular reprogramming', 'elite and stochastic models for induced pluripotent stem cell generation', 'transcription factors orchestrate dynamic interplay between genome topology and gene regulation during cell reprogramming', 'the circuitry of a master switch', 'a comprehensive library of human transcription factors for cell fate engineering', 'qiu et al. - 2022 - mapping transcriptomic vector fields of single cel', 'ahmad and henikoff - 2021 - the h3.3k27m oncohistone antagonizes reprogramming', 'conserved transcription factors promote cell fat

In [6]:
#Split into two methods?
def get_all_sources(prompt, path):
    prompt = prompt.lower()
    metadata_full = vectordb.get()['metadatas']
    source_list = [item['source'] for item in metadata_full]   
    real_source_list = [((item.replace(path, '')).removesuffix('.pdf')).lower() for item in source_list]
    db = pd.DataFrame({'id' :vectordb.get()['ids'] , 'metadatas' : real_source_list})
    filtered_df = db[db['metadatas'].apply(lambda x: x in prompt)]
    return real_source_list, filtered_df['id'].to_list()
        

In [7]:
prompt = 'Summarize the human transcription factors paper'
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
db_name = "new_DB_cosine_cSize_%d_cOver_%d" % (700, 200)
title_list, real_id_list = get_all_sources(prompt, path)
best_title, best_score = best_match(prompt, title_list)
title_list, real_id_list = get_all_sources(best_title, path)
text = vectordb.get(ids=real_id_list)['documents']
print(len(text))
#newdb = Chroma(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/new_EXP3/', embedding_function=embeddings_model, collection_name=db_name)
#print(len(newdb.get()['documents']))
#newdb.add_texts(text)
#print(len(newdb.get()['documents']))

#things to do - hook this up with RAG (shouldnt be too bad)
#determine how to sift thru the prompt or go thru paper titles that are not well done
#ask about which paper db to use on turbo
#where to save this new db
#sometimes there is a weird error that occurs - maybe split up lower text stuff but idk how that effects runtime
#how to remove docs from database / make new one everytime since it keeps adding to it

Wed Jun 19 09:35:12 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Wed Jun 19 09:35:14 2024 INFO Use pytorch device_name: cpu


NameError: name 'best_match' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer, util


#Given the prompt, find the title and corresponding score that is the best match
def best_match(prompt, title_list):
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    unique_title_list = list(set(title_list))
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(unique_title_list)

    save_title = ""
    save_score = 0

    for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
        if score > save_score:
            save_score = score
            save_title = title
    print(f"The best match is {save_title} with a score of {save_score}")
    return save_title, save_score

In [ ]:
save_title

In [ ]:
sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
unique_title_list = list(set(title_list))
query_embedding = sentence_model.encode(prompt)
passage_embedding = sentence_model.encode(unique_title_list)

save_title = ""
save_score = 0

for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
    if score > save_score:
        save_score = score
        save_title = title
print(f"The best match is {save_title} with a score of {save_score}")

In [ ]:
title_list, real_id_list = get_all_sources(save_title, "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/")
text = vectordb.get(ids=real_id_list)['documents']
#print(text)

In [ ]:


def restrictedDB(prompt, vectordb, path):
    #path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
    embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
    db_name = "new_DB_cosine_cSize_%d_cOver_%d" % (700, 200)
    title_list, real_id_list = get_all_sources(prompt, path)
    best_title, best_score = best_match(prompt, title_list)
    title_list, real_id_list = get_all_sources(best_title, path)
    text = vectordb.get(ids=real_id_list)['documents']
    newdb = Chroma(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/new_EXP3/', embedding_function=embeddings_model, collection_name=db_name)
    newdb.add_texts(text)
    return best_score, newdb


#Given the prompt, find the title and corresponding score that is the best match
def best_match(prompt, title_list):
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    unique_title_list = list(set(title_list))
    query_embedding = sentence_model.encode(prompt)
    passage_embedding = sentence_model.encode(unique_title_list)

    save_title = ""
    #adjust the save_score to be the threshold cutoff - set to 0.75 but maybe thats too high
    save_score = 0.75

    for score, title in zip(util.cos_sim(query_embedding, passage_embedding)[0], unique_title_list):
        if score > save_score:
            save_score = score
            save_title = title
    print(f"The best match is {save_title} with a score of {save_score}")
    return save_title, save_score

#Split into two methods?
def get_all_sources(prompt, path):
    prompt = prompt.lower()
    metadata_full = vectordb.get()['metadatas']
    source_list = [item['source'] for item in metadata_full]   
    real_source_list = [((item.replace(path, '')).removesuffix('.pdf')).lower() for item in source_list]
    db = pd.DataFrame({'id' :vectordb.get()['ids'] , 'metadatas' : real_source_list})
    filtered_df = db[db['metadatas'].apply(lambda x: x in prompt)]
    return real_source_list, filtered_df['id'].to_list()

In [ ]:
prompt = 'Summarize the human transcription factors paper'
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
#path = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/'
vectordb, embeddings_model = brad.load_literature_db(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/EXP2/')
best_score, text = restrictedDB(prompt, vectordb, path)
print(best_score)

## PageRanker 

In [ ]:
def remove_repeats(vectordb):
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents']})
    repeated_ids = df[df.duplicated(subset='documents', keep='last')]['id'].tolist()
    vectordb.delete(repeated_ids)
    return vectordb

In [ ]:
remove_repeats(vectordb)

In [ ]:
#vectordb.get()['documents']

In [23]:
#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(vectordb):
    dimension = len(vectordb.get()['documents'])
    adj_matrix = np.zeros([dimension, dimension])
    
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    passage_embedding = sentence_model.encode(vectordb.get()['documents'])
    cosine_similarities = cosine_similarity(passage_embedding[0:10])
    print(cosine_similarities)
    return cosine_similarities

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(vectordb):
    adj_matrix = adj_matrix_builder(vectordb)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents'], 'metadatas' : vectordb.get()['metadatas']})
    reranked_df = df.iloc[top_rank_scores]
    return reranked_df

In [24]:
rank_scores = pagerank_rerank(vectordb)
print(rank_scores)


Wed Jun 19 10:10:42 2024 INFO Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1
Wed Jun 19 10:10:43 2024 INFO Use pytorch device_name: cpu


Batches:   0%|          | 0/283 [00:00<?, ?it/s]

<class 'numpy.ndarray'>
[ 3.42600904e-02 -2.65194345e-02  4.13578190e-03 -6.21633492e-02
 -2.41281725e-02 -4.93638031e-02  2.85513680e-02  7.13860393e-02
 -2.77028400e-02 -3.23414840e-02  1.54855046e-02 -7.62076303e-02
 -2.67160474e-03 -8.98527075e-03 -1.44219518e-01 -3.77658121e-02
 -1.72684994e-02  5.09255640e-02 -1.00878596e-01  4.22701798e-02
  3.55484076e-02  2.83368211e-02  6.34182319e-02  1.38540920e-02
  4.28031944e-02 -2.33840593e-03 -5.75252585e-02 -3.45930345e-02
 -3.44563276e-02 -4.67588045e-02  3.03957816e-02  7.35652298e-02
 -1.88741330e-02  3.93743366e-02  9.98095945e-02  6.60319487e-03
  1.13752242e-02 -3.43076847e-02  5.02356282e-03  6.19645230e-02
  1.43595808e-03 -5.39184175e-02  1.55541557e-03  7.00846761e-02
 -1.05156219e-02  5.57598844e-02 -6.67627007e-02  2.79081725e-02
 -4.93292846e-02  2.93017998e-02 -2.24598031e-02 -1.59579709e-01
 -1.07039087e-01  7.74903372e-02 -5.58888307e-03  3.52224871e-03
 -3.88910174e-02 -8.47253352e-02  1.96575955e-03 -4.22144346e-02
 

In [ ]:
#first entry is always the prompt



In [10]:
#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(vectordb):
    dimension = len(vectordb.get()['documents'])
    adj_matrix = np.zeros([dimension, dimension])
    
    sentence_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
    passage_embedding = sentence_model.encode(vectordb.get()['documents'])
    print(type(passage_embedding))
    print(passage_embedding)
    pos = 1
    for score in zip(util.cos_sim(query_embedding, passage_embedding)[0]):
        adj_matrix[0][pos] = score[0]
        adj_matrix[pos][0] = score[0]
        pos += 1
    return adj_matrix

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(prompt, vectordb):
    adj_matrix = adj_matrix_builder(prompt, vectordb)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents'], 'metadatas' : vectordb.get()['metadatas']})
    reranked_df = df.iloc[top_rank_scores]
    return reranked_df

In [ ]:
#I THINK THIS FRAMEWORK WORKS BETTER


#Given the prompt, find the title and corresponding score that is the best match
def adj_matrix_builder(docs, scores):
    dimension = len(docs)
    adj_matrix = np.zeros([dimension, dimension])
    pos = 1
    for score in scores:
        adj_matrix[0][pos] = score
        adj_matrix[pos][0] = score
        pos += 1
    return adj_matrix

# Normalize columns of A
def normalize_adjacency_matrix(A):
    col_sums = A.sum(axis=0)
    return A / col_sums[np.newaxis, :]

#weighted pagerank algorithm
def pagerank_weighted(A, alpha=0.85, tol=1e-6, max_iter=100):
    n = A.shape[0]
    A_normalized = normalize_adjacency_matrix(A)
    v = np.ones(n) / n  # Initial PageRank vector

    for _ in range(max_iter):
        v_next = alpha * A_normalized.dot(v) + (1 - alpha) / n
        if np.linalg.norm(v_next - v, 1) < tol:
            break
        v = v_next

    return v

#reranker

def pagerank_rerank(docs, scores):
    adj_matrix = adj_matrix_builder(docs, scores)
    pagerank_scores = pagerank_weighted(A = adj_matrix)
    top_rank_scores = sorted(range(len(pagerank_scores)), key=lambda i: pagerank_scores[i], reverse=True)[1:11]
    reranked_docs = [docs[i] for i in top_rank_scores]
    return reranked_docs

In [ ]:
path = "/nfs/turbo/umms-indikar/shared/projects/RAG/papers/EXP2/"
metadata_full = vectordb.get()['metadatas']
source_list = [item['source'] for item in metadata_full]   
real_source_list = [((item.replace(path, '')).removesuffix('.pdf')).lower() for item in source_list]
df = pd.DataFrame({'id' :vectordb.get()['ids'] , 'documents' : vectordb.get()['documents'], 'metadatas' : real_source_list})
df

In [ ]:
brad.chat()

Pairwise Cosine Similarities:
[[1.         0.97463185 0.95941195]
 [0.97463185 1.         0.99819089]
 [0.95941195 0.99819089 1.        ]]


In [26]:
brad.chat()

Welcome to RAG! The chat log from this conversation will be saved to /home/machoi/BRAD/2024-06-19-10:27:27.230997.json. How can I help?

Would you like to use a database with BRAD [Y/N]?


 Y


Wed Jun 19 10:27:29 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/machoi/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Wed Jun 19 10:27:30 2024 INFO Use pytorch device_name: cpu
Wed Jun 19 10:27:30 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.
2024-06-19 10:27:31 INFO semantic_router.utils.logger local


NameError: name 'callMatlab' is not defined